In [12]:
from crewai import Task, Agent, Crew
from crewai_tools import JSONSearchTool

In [13]:
from src.utils import get_earnings_transcript
import re
import os
import json
from crewai_tools import JSONSearchTool

base_dir = "DATA"
os.makedirs(base_dir,exist_ok=True)
def clean_speakers(speaker):
    speaker = re.sub("\n", "", speaker)
    speaker = re.sub(":", "", speaker)
    return speaker

quarter = "Q1"
ticker = "AAPL"
year = "2023"

def get_save_earnings_data(quarter:str,ticker:str,year:str):
    ticker_path = os.path.join(base_dir,ticker)
    os.makedirs(ticker_path,exist_ok=True)
    ticker_year = os.path.join(ticker_path,year)
    os.makedirs(ticker_year,exist_ok=True)
    ticker_year_quarter = os.path.join(ticker_year,quarter)
    os.makedirs(ticker_year_quarter,exist_ok=True)

    resp_dict = get_earnings_transcript(quarter,ticker,year)

    content = resp_dict["content"]
    pattern = re.compile(r"\n(.*?):")
    matches = pattern.finditer(content)

    speakers_list = []
    ranges = []
    for match_ in matches:
        span_range = match_.span()
        ranges.append(span_range)
        speakers_list.append(match_.group())
    speakers_list = [clean_speakers(sl) for sl in speakers_list]

    earning_calls_docs = []
    for idx, speaker in enumerate(speakers_list[:-1]):
        start_range = ranges[idx][1]
        end_range = ranges[idx + 1][0]
        speaker_text = content[start_range + 1 : end_range]

        earning_calls_docs.append({"speaker":speaker,"quarter":quarter,"speaker_text":f"{speaker}: {speaker_text}"})

    earning_calls_docs.append({"speaker": speakers_list[-1], "quarter": quarter,'speaker_text':content[ranges[-1][1] :]})
    json_path = os.path.join(ticker_year_quarter,f'{quarter}.json')
    with open(json_path, 'w') as file:
        json.dump(earning_calls_docs, file, indent=4)
    json_tool = JSONSearchTool(json_path=json_path)
    return json_tool

In [17]:
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)

True

In [18]:
from tenacity import RetryError
from crewai_tools import JSONSearchTool  # Updated import path

print("Earnings Call Q1")

def get_quarter_tool(quarters,ticker,year):
    earnings_calls_tools = []
    for quarter in quarters:
        try:
            curr_quarter_tool = get_save_earnings_data(quarter,ticker,year)
            earnings_calls_tools.append(curr_quarter_tool)
        except RetryError:
            print(f"Don't have the data for {quarter}")
    return earnings_calls_tools
earnings_calls_tools = get_quarter_tool(["Q1","Q2","Q3","Q4"],ticker,year)


Earnings Call Q1


Inserting batches in chromadb: 100%|██████████| 1/1 [00:20<00:00, 20.74s/it]


In [26]:
from src.secData import sec_main
sec_docs,sec_form_names = sec_main(ticker=ticker,year=year)

Started Scraping
Scraped
Started Extracting
Extracted


In [33]:
for sec_filings,sec_name in zip(sec_docs,sec_form_names):
    report_year = sec_filings[-2].split("-")[0]
    if sec_name.startswith("10-Q"):
        curr_quarter = sec_name.split("-")[-1]
        save_path = os.path.join(base_dir,ticker,report_year,curr_quarter)
    else:
        save_path = os.path.join(base_dir,ticker,report_year)
    os.makedirs(save_path,exist_ok=True)
    with open(os.path.join(save_path,f'{sec_name}.json'), 'w') as file:
        json.dump(sec_filings[-1], file, indent=4)

In [19]:
from src.vectorDatabaseDocker import create_database

(
        qdrant_client,
        encoder,
        speakers_list_1,
        speakers_list_2,
        speakers_list_3,
        speakers_list_4,
        sec_form_names,
        earnings_call_quarter_vals,
    ) = create_database(ticker=ticker, year=int(float(year)))

Earnings Call Q1
Earnings Call Q2
Earnings Call Q3
Earnings Call Q4
SEC
Started Scraping
Scraped
Started Extracting
Extracted


/home/athekunal/Finance Data/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
